### Creating and Adding products to your localhost postgres database

In [3]:
import psycopg2
import psycopg2.extras
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [6]:
conn = psycopg2.connect(
	host="localhost",
	database="hackonwithamazon",
	user=os.getenv("USER"),
	password=os.getenv("PASSWORD")
)
try:
	curs = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

	curs.execute("CREATE TABLE products(id INTEGER PRIMARY KEY, img VARCHAR(30), price REAL, title VARCHAR(50), rating REAL)")

	curs.execute("INSERT INTO products VALUES(1, '1.jpeg', 24.65, 'Men Polyester Green T-Shirt', 4.1)")
	curs.execute("INSERT INTO products VALUES(2, '2.jpeg', 27.89, 'Cool Cargo Material Men Pant', 4.3)")
	curs.execute("INSERT INTO products VALUES(3, '3.jpeg', 18.25, 'Green Long Fit Pant', 4.6)")
	curs.execute("INSERT INTO products VALUES(4, '4.jpeg', 12.99, 'Comfortable Men Cargo Pant', 3.9)")
	curs.execute("INSERT INTO products VALUES(5, '5.jpeg', 30, 'Cargo Dark Green Pant', 4.3)")
	curs.execute("INSERT INTO products VALUES(6, '6.jpeg', 17.35, 'Green Men Cargo Pant', 4.1)")
	curs.execute("INSERT INTO products VALUES(7, '7.jpeg', 20.87, 'Navy Blue Men Pant', 4)")
	curs.execute("INSERT INTO products VALUES(8, '8.jpeg', 31.5, 'Women Full White Pant', 3.8)")
	curs.execute("INSERT INTO products VALUES(9, '9.jpeg', 29.65, 'Awesome Woolen Men Shirts', 4.5)")
	curs.execute("INSERT INTO products VALUES(10, '10.jpeg', 25.87, 'Super Cool Red Men Shirt', 4.2)")
	curs.execute("INSERT INTO products VALUES(11, '11.jpeg', 15.4, 'Pure White Shirt For Men', 4.3)")
	curs.execute("INSERT INTO products VALUES(12, '12.jpeg', 18.35, 'White Fullhand Men Shirt', 4.2)")
	curs.execute("INSERT INTO products VALUES(13, '13.jpeg', 24.20, 'Cool Modern White Shirt', 3.4)")
	curs.execute("INSERT INTO products VALUES(14, '14.jpeg', 26.80, 'Dark Blue Jacket', 3.9)")
	curs.execute("INSERT INTO products VALUES(15, '15.jpeg', 21.45, 'Pretty Blue Men Jacket', 4.5)")
	curs.execute("INSERT INTO products VALUES(16, '16.jpeg', 8.25, 'Military Cap', 4.1)")
	curs.execute("INSERT INTO products VALUES(17, '17.jpeg', 10, 'Small Military Cap', 4.7)")
	curs.execute("INSERT INTO products VALUES(18, '18.jpeg', 16.50, 'Green T-shirt', 4.0)")
	curs.execute("INSERT INTO products VALUES(19, '19.jpeg', 30.25, 'Pair of Green Tshirts', 3.4)")
	curs.execute("INSERT INTO products VALUES(20, '20.jpeg', 25, 'Floral Printed Dress', 4.3)")

	curs.close()
finally:
	conn.commit()
	conn.close()

### Creating pickle files for feature vectors and corresponding database ids

In [1]:
import pickle
import psycopg2
import psycopg2.extras
import os
import tensorflow as tf
from keras.applications.resnet50 import preprocess_input
import numpy as np
from numpy.linalg import norm
from dotenv import load_dotenv
from PIL import Image

In [2]:
load_dotenv()

True

In [3]:
model = tf.keras.models.load_model('similar_search/resnet50_model.keras')

In [4]:
def feature_extraction(image, model):
    image = image.resize((224, 224))
    image = np.array(image)
    pre_img = preprocess_input(np.expand_dims(image, axis=0))
    result = model.predict(pre_img).flatten()
    normalized = result / norm(result)
    return normalized

In [5]:
conn = psycopg2.connect(
    host="localhost",
    database="hackonwithamazon",
    user=os.getenv("USER"),
    password=os.getenv("PASSWORD")
)

feature_vectors = []
db_ids = []

try:
	curs = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

	curs.execute("SELECT id, img FROM products")
	results = curs.fetchall()
	for row in results:
		db_ids.append(row["id"])
		image = Image.open("database-imgs/"+row["img"])
		feature_vectors.append(feature_extraction(image, model))
	
	curs.close()
finally:
	conn.commit()
	conn.close()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


In [18]:
pickle.dump(feature_vectors, open('similar_search/feature_vectors.pkl', 'wb'))
pickle.dump(db_ids, open('similar_search/db_ids.pkl', 'wb'))